In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
from pandas.io.json import json_normalize
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
import plotly.io as pio
pio.renderers.default = 'plotly_mimetype+notebook'

In [ ]:
hotel = []
price = []
review = []
appreciation = []
adresse = []
experience = []

In [ ]:
page = 0
for i in range(1,10):
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    html = 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAQoggJCDXNlYXJjaF9mcmFuY2VIDVgEaE2IAQGYAQ24AQfIAQzYAQPoAQH4AQOIAgGoAgO4AoO0-IkGwAIB0gIkMGI5Y2FkZjctYzYzNC00NGE5LWFiMDgtNWM0YmU5N2Y1MTJj2AIE4AIB&tmpl=searchresults&checkin_month=10&checkin_monthday=5&checkin_year=2022&checkout_month=03&checkout_monthday=10&checkout_year=2022&class_interval=1&dest_id=73&dest_type=country&dtdisc=0&from_sf=2&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=5c796de6e7f4017e&ss=France&ss_all=0&ssb=empty&sshis=0&ssne=France&ssne_untouched=France&top_ufis=1&rows=25&offset={page}'
    page += 25
    response = requests.get(html,headers=headers)
    bsobj = soup(response.content, 'lxml')
    
    for name in bsobj.findAll('span',{'class':'sr-hotel__name'}):
        hotel.append(name.text.strip())

    for name in bsobj.findAll('span',{'class':'bui-u-sr-only'}):
        price.append(name.text.strip())


    for name in bsobj.findAll('div',{'class':'bui-review-score__badge'}):
        review.append(name.text.strip())


    for name in bsobj.findAll('div',{'class':'bui-review-score__title'}):
        appreciation.append(name.text.strip())


    for name in bsobj.findAll('div',{'class':'sr_card_address_line'}):
        adresse.append(name.text.strip())

    for name in bsobj.findAll('div',{'class':'bui-review-score__text'}):
        experience.append(name.text.strip())

In [ ]:
df_scrap_booking = pd.DataFrame(list(zip(hotel ,price, adresse, appreciation,review,experience)),
              columns=['name_hotel','price','adress_hotel', 'appreciation_hotel', 'review_hotel','nb_exp_vecu_hotel'])


In [ ]:
df_scrap_booking = pd.read_csv('Booking_sauv.csv')
del df_scrap_booking['Unnamed: 0']

In [ ]:
df_scrap_booking.adress_hotel = [col.replace("\nIndiquer sur la carte", "") for col in df_scrap_booking.adress_hotel]
df_scrap_booking.adress_hotel = [col.replace("\n\n", "") for col in df_scrap_booking.adress_hotel]
df_scrap_booking.adress_hotel = [col.replace("Proche du métro", "") for col in df_scrap_booking.adress_hotel]


In [ ]:
res = pd.DataFrame(df_scrap_booking.adress_hotel.str.split(",",1).tolist(),columns = ["indic","ville"])

res['ville']= res['ville'].fillna(res['indic'])
df_scrap_booking = pd.concat([df_scrap_booking, res], axis=1)

df_scrap_booking['ville'] = df_scrap_booking['ville'].str.strip()

df_scrap_booking['review_hotel'] = df_scrap_booking['review_hotel'].replace(',','.', regex=True).astype(float)

#### 1 - API météo Booking avec openweathermap

In [ ]:
df_meteo = pd.DataFrame(columns=['weather', 'base', 'visibility', 'dt', 'timezone', 'id', 'name', 'cod',
       'coord.lon', 'coord.lat', 'main.temp', 'main.feels_like',
       'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity',
       'wind.speed', 'wind.deg', 'clouds.all', 'sys.type', 'sys.id',
       'sys.country', 'sys.sunrise', 'sys.sunset'])

In [ ]:
villes = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar",
"Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille",
"Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege",
"Toulouse","Montauban","Biarritz","Bayonne",
"La Rochelle",'Chatou','Saint-Brice-en-Coglès','Saint-Raphaël','Kingersheim','Montargis','La Chapelle-de-Guinchay','Étampes','Bazoges-en-Paillers']

In [ ]:
for v in villes:
    # API 
    API_URL = "https://api.openweathermap.org/data/2.5/weather?"
    # API key
    API_KEY = ""
    # City 
    CITY = v
    # updating the URL
    URL = API_URL + "q=" + CITY + "&appid=" + API_KEY
    # Sending HTTP request
    response = requests.get(URL)
    ville_meteo = response.json()
    df_ville = pd.DataFrame.from_dict(json_normalize(ville_meteo), orient='columns')
    #result = pd.concat([df_meteo, df_ville], axis=1, join='inner')
    df_meteo = df_meteo.append(other=df_ville)
    


In [ ]:

cols = ['name','coord.lon','coord.lat','main.temp','main.humidity']
df_meteo = df_meteo[cols]
df_meteo = df_meteo[cols].dropna()


df_meteo['main.temp'] = df_meteo['main.temp']-273.15

df_meteo.name = [col.replace("Arrondissement de ", "") for col in df_meteo.name]
df_meteo.name = [col.replace("Arrondissement d'", "") for col in df_meteo.name]

In [ ]:
df_meteo.rename(columns={'name':'ville', 'coord.lon': 'longitude', 'coord.lat':'latitude', 'main.temp':'temperature', 'main.humidity':'tx_humidite'}, inplace=True)

In [ ]:
df_scrap_booking = pd.merge(df_scrap_booking , df_meteo, on='ville')

#### Top 5 des meilleurs destinations : note moyenne

In [ ]:
top5 = df_scrap_booking.groupby(['ville', 'latitude','longitude'])['review_hotel'].mean().sort_values(ascending=False)

top5.columns = ['review_mean']
top5 = top5.reset_index()
top5 = top5.iloc[0:5,1:5]

In [ ]:
import plotly.graph_objects as go

fig = px.scatter_mapbox(top5, lat="latitude", lon="longitude",                       
                        size_max=10, zoom=4, height=500, mapbox_style="carto-positron", color='review_hotel',
                        size="latitude",
                        title="TOP 5 des meilleurs destinations")



fig.show()

In [ ]:
top5_ville = df_scrap_booking.groupby(['latitude','ville'])['review_hotel'].mean().sort_values(ascending=False)
top5_ville = top5_ville.reset_index()
top5_ville = top5_ville.iloc[0:5,1:5]

print(top5_ville.ville)

#### Top-20 des hôtels 

In [ ]:
df_scrap_booking = df_scrap_booking.sort_values(by=['review_hotel'], ascending=False)

top_20 = df_scrap_booking.iloc[0:19,:]

In [ ]:
fig = px.scatter_mapbox(top_20, lat="latitude", lon="longitude",                       
                        size_max=10, zoom=4, height=600, mapbox_style="carto-positron", color='review_hotel',
                        size="latitude",
                        title="TOP 20 des meilleurs Hôtels")
 


fig.show()

In [ ]:

top_20 = df_scrap_booking.iloc[0:19,:]

top_20[['name_hotel','ville','appreciation_hotel','review_hotel']]